In [1]:
### PIPELINE: 0) read 1) preprocess 2) query and parse response 3) analyse

import pandas as pd
from dataclasses import dataclass


# define data structure for easier result parsing later on
@dataclass
class parsed_process_constraints:
    control_flow: int
    control_flow_in_activity: int
    data: int
    data_in_activity: int
    time: int
    time_in_activity: int
    resource: int
    resource_in_activity: int
    no_process_constraint: int


PATH_TO_PROMPT_TEMPLATE = "../prompt_few_shot.txt"  # replace with ../prompt_zero_shot.txt for a zero-shot approach.
# https://github.com/ollama/ollama/blob/main/docs/api.md temperature and seed for reproducibility
MODEL_TEMPERATURE = 0.1
MODEL_SEED = 1312

APPENDIX_A_REFERENCE = (
    "The activity complies with the criteria set out in Appendix A to this Annex."
)
APPENDIX_B_REFERENCE = (
    "The activity complies with the criteria set out in Appendix B to this Annex."
)
APPENDIX_C_REFERENCE = (
    "The activity complies with the criteria set out in Appendix C to this Annex."
)
APPENDIX_D_REFERENCE = (
    "The activity complies with the criteria set out in Appendix D to this Annex."
)
APPENDIX_E_REFERENCE = (
    "The activity complies with the criteria set out in Appendix E to this Annex."
)

SUBSTANTIAL_CONTRIBUTION_CRITERIA = "Substantial contribution criteria"
DNSH_CLIMATE_ADAPTION = "DNSH on Climate adaptation"
DNSH_CLIMATE_MITIGATION = "DNSH on Climate mitigation"
DNSH_WATER = "DNSH on Water"
DNSH_CIRCULAR_ECONOMY = "DNSH on Circular economy"
DNSH_POLLUTION = "DNSH on Pollution prevention"
DNSH_BIODIVERSITY = "DNSH on Biodiversity"

In [2]:
# Following prompting from https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Prompt_Engineering_with_Llama_3.ipynb
import os
from typing import Dict, List
from groq import Groq
from dotenv import load_dotenv


# Get a free API key from https://console.groq.com/keys, and put it into .env in the top level of this repo
load_dotenv('../')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_KEY')

# Defines the exact model to be used. Here, we use the one fine-tuned for following instructions.
LLAMA3_70B_INSTRUCT = "llama3-70b-8192"
LLAMA3_8B_INSTRUCT = "llama3-8b-8192"


client = Groq()

def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def chat_completion(
    messages: List[Dict],
    model = LLAMA3_8B_INSTRUCT,
    temperature: float = MODEL_TEMPERATURE, # for quasi-deterministic responses
    top_p: float = 0.1, # for quasi-deterministic responses
    seed = MODEL_SEED, # for quasi-deterministic responses
) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
        seed=seed,
    )
    return response.choices[0].message.content  

def completion(
    prompt: str,
    model: str = LLAMA3_8B_INSTRUCT,
    temperature: float = MODEL_TEMPERATURE, # for quasi-deterministic responses
    top_p: float = 0.1, # for quasi-deterministic responses
    seed = MODEL_SEED, # for quasi-deterministic responses
) -> str:
    return chat_completion(
        [user(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
        seed=seed,
    )

#def complete_and_print(prompt: str, model: str = LLAMA3_8B_INSTRUCT):
#    print(f'==============\n{prompt}\n==============')
#    response = completion(prompt, model)
#    print(response, end='\n\n')

In [4]:
#### PIPELINE Stage 0: Read Files #####

from pathlib import Path

# Parse environmental objective catalogues
climate_mitigation = pd.read_excel('../taxonomy/taxonomy.xlsx', sheet_name="Climate mitigation")
climate_adaption = pd.read_excel('../taxonomy/taxonomy.xlsx', sheet_name="Climate adaptation")
water = pd.read_excel('../taxonomy/taxonomy.xlsx', sheet_name="Water")
circular_economy = pd.read_excel('../taxonomy/taxonomy.xlsx', sheet_name="Circular economy")
pollution_prevention = pd.read_excel('../taxonomy/taxonomy.xlsx', sheet_name="Pollution prevention")
biodiversity = pd.read_excel('../taxonomy/taxonomy.xlsx', sheet_name="Biodiversity")

# Read texts of taxonomy appendices
appendix_a = Path("../taxonomy/appendix_a").read_text()
appendix_b = Path("../taxonomy/appendix_b").read_text()
appendix_c = Path("../taxonomy/appendix_c").read_text()
appendix_d = Path("../taxonomy/appendix_d").read_text()
appendix_e = Path("../taxonomy/appendix_e").read_text()

In [5]:
#### PIPELINE Stage 1: Preprocess Catalogues #####

# Preprocess catalogues by 1) filtering out N/As, 2) re-adding footnotes

from numpy import NaN
import re

criteria = [
    SUBSTANTIAL_CONTRIBUTION_CRITERIA,
    DNSH_CLIMATE_ADAPTION,
    DNSH_CLIMATE_MITIGATION,
    DNSH_WATER,
    DNSH_CIRCULAR_ECONOMY,
    DNSH_POLLUTION,
    DNSH_BIODIVERSITY,
]

def filter_nan(s):
    if s == "N/A" or s == "N/A ":
        return None
    else:
        return s


def preprocess_row(row):
    # filter out N/A and replace with None
    row[:] = row[:].apply(filter_nan)

    if row["Footnotes"] is not None and row["Footnotes"] is not NaN:
        # Add footnote to appropriate criteria chunks, should they contain footnotes
        for single_criteria in criteria:
            try:
                if row[single_criteria] is not None and isinstance(
                    row[single_criteria], str
                ):
                    footnotes_in_single_criteria = re.search(
                        r"\(+[0-9]\)", row[single_criteria]
                    )
                    if footnotes_in_single_criteria is not None:
                        # append footnotes to chunk
                        row[single_criteria] = (
                            row[single_criteria] + "\n\nHere are additional footnotes for the text passage:\n" + row["Footnotes"]
                        )
            except KeyError:
                continue
    return row


climate_mitigation_preprocessed = climate_mitigation.apply(
    lambda row: preprocess_row(row), axis=1
)
climate_adaption_preprocessed = climate_adaption.apply(
    lambda row: preprocess_row(row), axis=1
)
water_preprocessed = water.apply(lambda row: preprocess_row(row), axis=1)
circular_economy_preprocessed = circular_economy.apply(
    lambda row: preprocess_row(row), axis=1
)
pollution_prevention_preprocessed = pollution_prevention.apply(
    lambda row: preprocess_row(row), axis=1
)
biodiversity_preprocessed = biodiversity.apply(lambda row: preprocess_row(row), axis=1)

In [6]:
#### PIPELINE Stage 2 + 3: Query LLM per catalogue, row, criteria (chunk) and parse response #####
import time
import ast
from typing import Optional
import regex
from groq import RateLimitError, InternalServerError
from tenacity import (
    retry,
    stop_after_attempt,
    retry_if_exception_type,
    wait_exponential,
    retry_if_result,
)


def log_before_prompt_retry(retry_state):
    if retry_state.outcome.failed:
        print(
            "retrying attempt "
            + str(retry_state.attempt_number)
            + ", ended with exception:"
            + str(retry_state.outcome)
        )
    else:
        print(
            "retrying attempt "
            + str(retry_state.attempt_number)
            + ", ended with "
            + str(retry_state.outcome.result())
        )


def is_none_p(value):
    """Return True if process constraints value is None"""
    return value[0] is None


criteria = [
    SUBSTANTIAL_CONTRIBUTION_CRITERIA,
    DNSH_CLIMATE_ADAPTION,
    DNSH_CLIMATE_MITIGATION,
    DNSH_WATER,
    DNSH_CIRCULAR_ECONOMY,
    DNSH_POLLUTION,
    DNSH_BIODIVERSITY,
]

APPENDIX_A_CRITERIA = None
APPENDIX_A_RESPONSE = None
APPENDIX_B_CRITERIA = None
APPENDIX_B_RESPONSE = None
APPENDIX_C_CRITERIA = None
APPENDIX_C_RESPONSE = None
APPENDIX_D_CRITERIA = None
APPENDIX_D_RESPONSE = None
APPENDIX_E_CRITERIA = None
APPENDIX_E_RESPONSE = None

# per catalogue: per substantial contribution, DNSH: query model with prompt template
from pathlib import Path

prompt_template = Path(PATH_TO_PROMPT_TEMPLATE).read_text()


# Insert chunk of taxonomy into prepared prompt
def create_prompt_from_preprocessed_chunk(
    chunk: str, activity_description: Optional[str], template: str
):
    # we opt to not add references to existing standards or legislation here, as it's safe to assume (checked with some samples) that these texts are part of the LLM's training data.
    # TODO: look at https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Prompt_Engineering_with_Llama_3.ipynb for better few-shot prompting with assistant messages

    if activity_description is None:
        return template.format(description_of_activity="", taxonomy_chunk=chunk)
    else:
        description = (
            "\nHere is a high-level description of the process that is regulated by the text passage you will be given later on:\n"
            + activity_description
        )
        return template.format(
            description_of_activity=description, taxonomy_chunk=chunk
        )


# Query model with prepared prompt, and parse response into constraints
# Retry in case a rate limit has been hit or no constraints were present in the response, with an increasing waiting time
@retry(
    stop=stop_after_attempt(30),
    retry=(
        retry_if_exception_type(tuple((RateLimitError, InternalServerError)))
        | retry_if_result(is_none_p)
    ),
    wait=wait_exponential(multiplier=1, min=4, max=900),
    before_sleep=log_before_prompt_retry,
)
def query_and_parse_response_for_chunk_prompt(
    chunk_prompt: str,
) -> tuple[Optional[parsed_process_constraints], str]:
    # print(chunk_prompt)

    response = completion(chunk_prompt)

    """ The response should contain an object-like string which we want to parse:
    {
    'control-flow': {
        'within_activities': [number of activity existence constraints],
        'between_activities': [number of control-flow constraints between activities]
    },
    'temporal': {
        'within_activities': [number of temporal constraints within activities],
        'between_activities': [number of temporal constraints between activities]
    },
    'resource': {
        'within_activities': [number of resource constraints within activities],
        'between_activities': [number of resource constraints between activities]
    },
    'data':{
        'within_activities': [number of data constraints within activities],
        'between_activities': [number of data constraints between activities]
    },
    'irrelevant': [number of irrelevant constraints]
    }

    """

    matched_response = regex.search(r"{((?:[^{}]|(?R))*)}", response, regex.VERBOSE)
    if matched_response is None or matched_response[0] == "":
        print("Found no match!")
        print(response)
        return None, response
    # print("match! parsing now...")
    preprocessed_response = matched_response[0].replace(
        "//", "#"
    )  # replace comment-like symbol with an actual comment
    preprocessed_response = preprocessed_response.replace(
        "[", ""
    )  # replace list-like symbol
    preprocessed_response = preprocessed_response.replace(
        "]", ""
    )  # replace list-like symbol

    parsed_response = ast.literal_eval(preprocessed_response)
    return (
        parsed_process_constraints(
            control_flow=parsed_response["control-flow"]["between_activities"],
            control_flow_in_activity=parsed_response["control-flow"][
                "within_activities" # this is actually an activity existence constraint
            ],
            time=parsed_response["temporal"]["between_activities"],
            time_in_activity=parsed_response["control-flow"]["between_activities"],
            resource=parsed_response["resource"]["between_activities"],
            resource_in_activity=parsed_response["resource"]["within_activities"],
            data=parsed_response["data"]["between_activities"],
            data_in_activity=parsed_response["data"]["within_activities"],
            no_process_constraint=parsed_response["irrelevant"],
        ),
        response,
    )


# Create prompt for chunk and then prompt and return parsed result
def prompt_for_criteria(
    criteria: str, description: Optional[str]
) -> tuple[parsed_process_constraints, str]:
    # create prompt for a specific DNSH/contribution criteria

    prompt = create_prompt_from_preprocessed_chunk(
        criteria, description, prompt_template
    )
    # sleep 2 seconds to not hit rate limit of Groq
    time.sleep(2)

    # return parsed result of querying LLM. Prompt is re-done automatically until we get a proper response
    return query_and_parse_response_for_chunk_prompt(prompt) # type: ignore
    

# Prompt model for a specific criteria of a taxonomy row
def prompt_for_criteria_of_row_if_exists(
    criteria: str, description: Optional[str], row
):
    try:
        if row[criteria] is not None and row[criteria] is not NaN:
            criteria_of_row: str = row[criteria]
            # Use previously generated results for appendices, if they solely referenced
            if criteria_of_row in APPENDIX_A_REFERENCE:
                return APPENDIX_A_CRITERIA, APPENDIX_A_RESPONSE
            elif criteria_of_row in APPENDIX_B_REFERENCE:
                return APPENDIX_B_CRITERIA, APPENDIX_B_RESPONSE
            elif criteria_of_row in APPENDIX_C_REFERENCE:
                return APPENDIX_C_CRITERIA, APPENDIX_C_RESPONSE
            elif criteria_of_row in APPENDIX_D_REFERENCE:
                return APPENDIX_D_CRITERIA, APPENDIX_D_RESPONSE
            elif criteria_of_row in APPENDIX_E_REFERENCE:
                return APPENDIX_E_CRITERIA, APPENDIX_E_RESPONSE
            # Otherwise, prompt model directly
            else:
                # if only partly referenced, add appendix to criteria
                if "Appendix A" in criteria_of_row:
                    criteria_of_row += "\n\n Appendix A:\n" + appendix_a
                if "Appendix B" in criteria_of_row:
                    criteria_of_row += "\n\n Appendix B:\n" + appendix_b
                if "Appendix C" in criteria_of_row:
                    criteria_of_row += "\n\n Appendix C:\n" + appendix_c
                if "Appendix D" in criteria_of_row:
                    criteria_of_row += "\n\n Appendix D:\n" + appendix_d
                if "Appendix E" in criteria_of_row:
                    criteria_of_row += "\n\n Appendix E:\n" + appendix_e
                constraints_of_criteria_row, raw_response = prompt_for_criteria(
                    criteria_of_row, description=description
                )
        else:
            constraints_of_criteria_row = None
            raw_response = ""
    except KeyError:
        constraints_of_criteria_row = None
        raw_response = ""
    return constraints_of_criteria_row, raw_response


def query_row_of_catalogue(goal: str, row):
    description = row["Description"]

    substantial_contribution = row[SUBSTANTIAL_CONTRIBUTION_CRITERIA]
    substantial_contribution_constraints, substantial_contribution_response = (
        prompt_for_criteria(substantial_contribution, description)
    )

    climate_mitigation_dnsh_constraints, climate_mitigation_response = (
        prompt_for_criteria_of_row_if_exists(DNSH_CLIMATE_MITIGATION, description, row)
    )
    climate_adaption_dnsh_constraints, climate_adaption_response = (
        prompt_for_criteria_of_row_if_exists(DNSH_CLIMATE_ADAPTION, description, row)
    )
    water_dnsh_constraints, water_response = prompt_for_criteria_of_row_if_exists(
        DNSH_WATER, description, row
    )
    circular_dnsh_constraints, circular_response = prompt_for_criteria_of_row_if_exists(
        DNSH_CIRCULAR_ECONOMY, description, row
    )
    pollution_dnsh_constraints, pollution_response = (
        prompt_for_criteria_of_row_if_exists(DNSH_POLLUTION, description, row)
    )

    biodiversity_dnsh_constraints, biodiversity_response = (
        prompt_for_criteria_of_row_if_exists(DNSH_BIODIVERSITY, description, row)
    )

    if row["NACE"] is not None and isinstance(row["NACE"], str):
        nace_codes = row["NACE"].split(",")
    else:
        nace_codes = []

    if row["Contribution type"] is not None and row["Contribution type"] is not NaN:
        contribution_type = row["Contribution type"]
    else:
        contribution_type = None

    row["goal"] = goal
    row["nace"] = nace_codes
    row["sector"] = row["Sector"]
    row["number"] = row["Activity number"]
    row["activity"] = row["Activity"]
    row["substantial"] = substantial_contribution_constraints
    row["mitigation"] = climate_mitigation_dnsh_constraints
    row["adaption"] = climate_adaption_dnsh_constraints
    row["water"] = water_dnsh_constraints
    row["circular"] = circular_dnsh_constraints
    row["pollution"] = pollution_dnsh_constraints
    row["biodiversity"] = biodiversity_dnsh_constraints
    row["contribution"] = contribution_type
    row["substantial_response"] = substantial_contribution_response
    row["mitigation_response"] = climate_mitigation_response
    row["adaption_response"] = climate_adaption_response
    row["water_response"] = water_response
    row["circular_response"] = circular_response
    row["pollution_response"] = pollution_response
    row["biodiversity_response"] = biodiversity_response
    return row[
        [
            "number",
            "nace",
            "goal",
            "sector",
            "activity",
            "substantial",
            "mitigation",
            "adaption",
            "water",
            "circular",
            "pollution",
            "biodiversity",
            "contribution",
            "substantial_response",
            "mitigation_response",
            "adaption_response",
            "water_response",
            "circular_response",
            "pollution_response",
            "biodiversity_response"
        ]
    ]


# Prepare appendix chunks by prompting for them once
APPENDIX_A_CRITERIA, APPENDIX_A_RESPONSE = prompt_for_criteria(appendix_a, "Take into account all relevant and irrelevant constraints you can identify in the text below.")
APPENDIX_B_CRITERIA, APPENDIX_B_RESPONSE = prompt_for_criteria(appendix_b, "Take into account all relevant and irrelevant constraints you can identify in the text below.")
APPENDIX_C_CRITERIA, APPENDIX_C_RESPONSE = prompt_for_criteria(appendix_c, "Take into account all relevant and irrelevant constraints you can identify in the text below.")
APPENDIX_D_CRITERIA, APPENDIX_D_RESPONSE = prompt_for_criteria(appendix_d, "Take into account all relevant and irrelevant constraints you can identify in the text below.")
APPENDIX_E_CRITERIA, APPENDIX_E_RESPONSE = prompt_for_criteria(appendix_e, "Take into account all relevant and irrelevant constraints you can identify in the text below. In this case, technical specifications can be interpreted as data constraints of general activities that deal with water appliances.")

In [7]:
# Apply prompt to all tables, all constraint cells of taxonomy


def apply_prompt(df, goal):
    df = df.apply(lambda row: query_row_of_catalogue(goal, row), axis=1)
    return df

water_parsed = apply_prompt(water_preprocessed, "Water")
water_parsed.to_csv(
    "../output/water_" + time.strftime("%Y%m%d-%H%M%S") + ".csv", sep="|"
)

circular_economy_parsed = apply_prompt(
    circular_economy_preprocessed, "Circular economy"
)
circular_economy_parsed.to_csv(
    "../output/circular_economy_" + time.strftime("%Y%m%d-%H%M%S") + ".csv", sep="|"
)

pollution_prevention_parsed = apply_prompt(
    pollution_prevention_preprocessed, "Pollution prevention"
)
pollution_prevention_parsed.to_csv(
    "../output/pollution_prevention_" + time.strftime("%Y%m%d-%H%M%S") + ".csv", sep="|"
)

biodiversity_parsed = apply_prompt(biodiversity_preprocessed, "Biodiversity")
biodiversity_parsed.to_csv(
    "../output/biodiversity_" + time.strftime("%Y%m%d-%H%M%S") + ".csv", sep="|"
)

climate_mitigation_parsed = apply_prompt(
    climate_mitigation_preprocessed, "Climate mitigation"
)
climate_mitigation_parsed.to_csv(
    "../output/climate_mitigation_" + time.strftime("%Y%m%d-%H%M%S") + ".csv", sep="|"
)

climate_adaption_parsed = apply_prompt(
    climate_adaption_preprocessed, "Climate adaption"
)
climate_adaption_parsed.to_csv(
    "../output/climate_adaption_" + time.strftime("%Y%m%d-%H%M%S") + ".csv", sep="|"
)

In [8]:
# Flatten each taxonomy data frame to have columns per contribution goal and constraint type


def flatten_element(element):
    # Make sure that we only have ints in there
    if type(element) is list:
        return int(element[0])
    else:
        return int(element)

def flatten_contribution(row, criteria):
    process_constraints: parsed_process_constraints = row[criteria]
    # Expand parsed process constraints into single columns
    if process_constraints is not None:
        row[criteria + "_control_flow_between"] = flatten_element(
            process_constraints.control_flow
        )
        row[criteria + "_control_flow_within"] = flatten_element(
            process_constraints.control_flow_in_activity # this is actually referring to activity existence constraint
        )
        row[criteria + "_temporal_between"] = flatten_element(process_constraints.time)
        row[criteria + "_temporal_within"] = flatten_element(
            process_constraints.time_in_activity
        )
        row[criteria + "_resource_between"] = flatten_element(
            process_constraints.resource
        )
        row[criteria + "_resource_within"] = flatten_element(
            process_constraints.resource_in_activity
        )
        row[criteria + "_data_between"] = flatten_element(process_constraints.data)
        row[criteria + "_data_within"] = flatten_element(
            process_constraints.data_in_activity
        )
        row[criteria + "_no_process_relation"] = flatten_element(
            process_constraints.no_process_constraint
        )
    else:
        row[criteria + "_control_flow_between"] = 0
        row[criteria + "_control_flow_within"] = 0 # this is actually referring to activity existence constraint
        row[criteria + "_temporal_between"] = 0
        row[criteria + "_temporal_within"] = 0
        row[criteria + "_resource_between"] = 0
        row[criteria + "_resource_within"] = 0
        row[criteria + "_data_between"] = 0
        row[criteria + "_data_within"] = 0
        row[criteria + "_no_process_relation"] = 0
    return row


def flatten_row_of_taxonomy(row):
    row = flatten_contribution(row, "substantial")
    row = flatten_contribution(row, "mitigation")
    row = flatten_contribution(row, "adaption")
    row = flatten_contribution(row, "water")
    row = flatten_contribution(row, "circular")
    row = flatten_contribution(row, "pollution")
    row = flatten_contribution(row, "biodiversity")
    # Drop columns we do no longer need for direct analysis and have exported to csv in a previous step
    row = row.drop(
        [
            "substantial",
            "mitigation",
            "adaption",
            "water",
            "circular",
            "pollution",
            "biodiversity",
            "contribution",
            "substantial_response",
            "mitigation_response",
            "adaption_response",
            "water_response",
            "circular_response",
            "pollution_response",
            "biodiversity_response"
        ]
    )
    return row


def flatten_taxonomy(df):
    df = df.apply(lambda row: flatten_row_of_taxonomy(row), axis=1)
    return df

output_time = time.strftime("%Y%m%d-%H%M%S")

climate_mitigation_flattened = flatten_taxonomy(climate_mitigation_parsed)
climate_mitigation_flattened.to_csv(
    "../output/climate_mitigation_flattened_" + output_time + ".csv",
    sep="|",
)

climate_adaption_flattened = flatten_taxonomy(climate_adaption_parsed)
climate_adaption_flattened.to_csv(
    "../output/climate_adaption_flattened_" + output_time + ".csv",
    sep="|",
)

water_flattened = flatten_taxonomy(water_parsed)
water_flattened.to_csv(
    "../output/water_flattened_" + output_time + ".csv", sep="|"
)

circular_economy_flattened = flatten_taxonomy(circular_economy_parsed)
circular_economy_flattened.to_csv(
    "../output/circular_economy_flattened_" + output_time + ".csv",
    sep="|",
)

pollution_prevention_flattened = flatten_taxonomy(pollution_prevention_parsed)
pollution_prevention_flattened.to_csv(
    "../output/pollution_prevention_flattened_"
    + output_time
    + ".csv",
    sep="|",
)

biodiversity_flattened = flatten_taxonomy(biodiversity_parsed)
biodiversity_flattened.to_csv(
    "../output/biodiversity_flattened_" + output_time + ".csv",
    sep="|",
)

In [27]:
# Make results available in second notebook for further analysis

climate_mitigation_flattened.to_hdf('../output/climate_mitigation_flattened.h5', key='random_data')
climate_adaption_flattened.to_hdf('../output/climate_adaption_flattened.h5', key='random_data')
water_flattened.to_hdf('../output/water_flattened.h5', key='random_data')
circular_economy_flattened.to_hdf('../output/circular_economy_flattened.h5',key='random_data')
pollution_prevention_flattened.to_hdf('../output/pollution_prevention_flattened.h5',key='random_data')
biodiversity_flattened.to_hdf('../output/biodiversity_flattened.h5',key='random_data')

C:\Users\Finn\AppData\Local\Temp\ipykernel_10616\164278636.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['nace', 'goal', 'sector', 'activity'], dtype='object')]

  climate_mitigation_flattened.to_hdf('../output/climate_mitigation_flattened.h5', key='random_data')
C:\Users\Finn\AppData\Local\Temp\ipykernel_10616\164278636.py:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['nace', 'goal', 'sector', 'activity'], dtype='object')]

  climate_adaption_flattened.to_hdf('../output/climate_adaption_flattened.h5', key='random_data')
C:\Users\Finn\AppData\Local\Temp\ipykernel_10616\164278636.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [i

In [10]:
# Store each constraint set and their concatination in an excel, as well as the raw output for further inspection

concatinated_taxonomy = pd.concat([climate_mitigation_flattened, climate_adaption_flattened, water_flattened, circular_economy_flattened, pollution_prevention_flattened, biodiversity_flattened], axis=0)

with pd.ExcelWriter('../output/output_' + output_time + '.xlsx') as writer:
    concatinated_taxonomy.to_excel(writer, "Overall results")
    climate_mitigation_flattened.to_excel(writer, "Climate mitigation")
    climate_adaption_flattened.to_excel(writer, "Climate adaption")
    water_flattened.to_excel(writer, "Water")
    circular_economy_flattened.to_excel(writer, "Circular economy")
    pollution_prevention_flattened.to_excel(writer, "Pollution prevention")
    biodiversity_flattened.to_excel(writer, "Biodiversity")


with pd.ExcelWriter('../output/output_raw_' + output_time + '.xlsx') as writer:
    climate_mitigation_parsed.to_excel(writer, "Climate mitigation")
    climate_adaption_parsed.to_excel(writer, "Climate adaption")
    water_parsed.to_excel(writer, "Water")
    circular_economy_parsed.to_excel(writer, "Circular economy")
    pollution_prevention_parsed.to_excel(writer, "Pollution prevention")
    biodiversity_parsed.to_excel(writer, "Biodiversity")


C:\Users\Finn\AppData\Local\Temp\ipykernel_10616\2173629367.py:6: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  concatinated_taxonomy.to_excel(writer, "Overall results")
C:\Users\Finn\AppData\Local\Temp\ipykernel_10616\2173629367.py:7: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  climate_mitigation_flattened.to_excel(writer, "Climate mitigation")
C:\Users\Finn\AppData\Local\Temp\ipykernel_10616\2173629367.py:8: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  climate_adaption_flattened.to_excel(writer, "Climate adaption")
C:\Users\Finn\AppData\Local\Temp\ipykernel_10616\2173629367.py:9: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
